## Bayesian Network Student Example

<img src="images/Network_Student.png" width="450">

In [41]:
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination
# Defining the model structure. We can define the network by just passing a list of edges.
model = BayesianModel([('D', 'G'), ('I', 'G'), ('G', 'L'), ('I', 'S')])
# Defining individual CPDs.

cpd_D = TabularCPD('D',
                    2,
                    [[0.6,0.4]])

cpd_I = TabularCPD('I',
                    2,
                    [[0.7,0.3]])

# The representation of CPD in pgmpy is a bit different than the CPD shown in the above picture. In pgmpy the colums
# are the evidences and rows are the states of the variable. So the grade CPD is represented like this:
#
#    +---------+---------+---------+---------+---------+
#    | diff    | intel_0 | intel_0 | intel_1 | intel_1 |
#    +---------+---------+---------+---------+---------+
#    | intel   | diff_0  | diff_1  | diff_0  | diff_1  |
#    +---------+---------+---------+---------+---------+
#    | grade_0 | 0.3     | 0.05    | 0.9     | 0.5     |
#    +---------+---------+---------+---------+---------+
#    | grade_1 | 0.4     | 0.25    | 0.08    | 0.3     |
#    +---------+---------+---------+---------+---------+
#    | grade_2 | 0.3     | 0.7     | 0.02    | 0.2     |
#    +---------+---------+---------+---------+---------+
cpd_S = TabularCPD('S',
                    2,
                    [[0.95, 0.2],
                     [0.05, 0.8]],
                    evidence = ['I'],
                    evidence_card=[2])

cpd_G = TabularCPD('G',
                    3,
                    [[ 0.3,   0.05,  0.9,   0.5 ],
                     [ 0.4,   0.25,  0.08,  0.3 ],
                     [ 0.3,   0.7,   0.02,  0.2 ]],
                    evidence = ['I','D'],
                    evidence_card=[2,2])

cpd_L = TabularCPD('L',
                    2,
                    [[ 0.1,   0.4,   0.99],
                     [ 0.9,   0.6,   0.01]],
                    evidence = ['G'],
                    evidence_card=[3])

model.add_cpds(cpd_D,
               cpd_I,
               cpd_S,
               cpd_G,
               cpd_L)

model.check_model()

infer = VariableElimination(model)

print(infer.query(['L'], evidence={'I': 0}))
print(infer.query(['L'], evidence={'I': 0, 'D': 0}))


Eliminating: S: 100%|██████████| 2/2 [00:00<00:00, 1072.44it/s]

+------+----------+
| L    |   phi(L) |
+======+==========+
| L(0) |   0.6114 |
+------+----------+
| L(1) |   0.3886 |
+------+----------+
+------+----------+
| L    |   phi(L) |
+======+==========+
| L(0) |   0.4870 |
+------+----------+
| L(1) |   0.5130 |
+------+----------+


In [42]:
print(cpd_G)

+------+------+------+------+------+
| I    | I(0) | I(0) | I(1) | I(1) |
+------+------+------+------+------+
| D    | D(0) | D(1) | D(0) | D(1) |
+------+------+------+------+------+
| G(0) | 0.3  | 0.05 | 0.9  | 0.5  |
+------+------+------+------+------+
| G(1) | 0.4  | 0.25 | 0.08 | 0.3  |
+------+------+------+------+------+
| G(2) | 0.3  | 0.7  | 0.02 | 0.2  |
+------+------+------+------+------+


In [43]:
from pgmpy.factors.discrete import TabularCPD

cpd_D = TabularCPD('D',
                    2,
                    [[0.6,0.4]])

cpd_I = TabularCPD('I',
                    2,
                    [[0.7,0.3]])

cpd_G = TabularCPD('G',
                    3,
                    [[ 0.3,   0.05,  0.9,   0.5 ],
                     [ 0.4,   0.25,  0.08,  0.3 ],
                     [ 0.3,   0.7,   0.02,  0.2 ]],
                    evidence = ['I','D'],
                    evidence_card=[2,2])

P = cpd_G*cpd_D*cpd_I
print(P)

print('\nP(D=1 | G=2) / P(G=2) = (0.196 + 0.024) / (0.126 + 0.196 + 0.0036 + 0.024)\n')
print ((0.196 + 0.024) / (0.126 + 0.196 + 0.0036 + 0.024))

+------+---------------------+----------------------+--------+----------------------+
| I    | I(0)                | I(0)                 | I(1)   | I(1)                 |
+------+---------------------+----------------------+--------+----------------------+
| D    | D(0)                | D(1)                 | D(0)   | D(1)                 |
+------+---------------------+----------------------+--------+----------------------+
| G(0) | 0.126               | 0.014000000000000002 | 0.162  | 0.06                 |
+------+---------------------+----------------------+--------+----------------------+
| G(1) | 0.16799999999999998 | 0.06999999999999999  | 0.0144 | 0.036                |
+------+---------------------+----------------------+--------+----------------------+
| G(2) | 0.126               | 0.19599999999999998  | 0.0036 | 0.024000000000000004 |
+------+---------------------+----------------------+--------+----------------------+

P(D=1 | G=2) / P(G=2) = (0.196 + 0.024) / (0.126 + 0.